In [39]:
import pandas as pd
import numpy as np
import datetime
import os
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [40]:
price_2020 = pd.read_csv('Price_2020.csv')
price_2021 = pd.read_csv('Price_2021.csv')
price_2022 = pd.read_csv('Price_2022.csv')

demand_2020 = pd.read_csv('Demand_2020.csv')
demand_2021 = pd.read_csv('Demand_2021.csv')
demand_2022 = pd.read_csv('Demand_2022.csv')

In [41]:
price = pd.concat([price_2020,price_2021]).reset_index().drop(columns={'index'})

In [42]:
price = pd.concat([price,price_2022]).reset_index().drop(columns={'index'})

In [43]:
price.head()

,Date,Hour,HOEP,Hour 1 Predispatch,Hour 2 Predispatch,Hour 3 Predispatch,OR 10 Min Sync,OR 10 Min non-sync,OR 30 Min
0,2020-01-01,1,0.0,0.0,0.0,0.00,1.03,0.20,0.20
1,2020-01-01,2,0.0,0.0,0.0,0.00,1.04,0.20,0.20
2,2020-01-01,3,0.0,0.0,0.0,0.00,1.05,0.20,0.20
3,2020-01-01,4,0.0,0.0,0.0,-0.02,1.06,0.18,0.18
4,2020-01-01,5,0.0,0.0,0.0,-0.10,1.07,0.20,0.20


In [44]:
demand = pd.concat([demand_2020,demand_2021]).reset_index().drop(columns={'index'})

In [45]:
demand = pd.concat([demand,demand_2022]).reset_index().drop(columns={'index'})

In [46]:
demand.head()

,Date,Hour,Market Demand,Ontario Demand
0,2020-01-01,1,16565,13219
1,2020-01-01,2,16193,12895
2,2020-01-01,3,16072,12554
3,2020-01-01,4,15930,12360
4,2020-01-01,5,15963,12353


In [47]:
df = pd.merge(price,demand,on=['Date','Hour'],how = 'left')

In [48]:
df = df[['Date','Hour','HOEP','Hour 1 Predispatch','Hour 2 Predispatch','Hour 3 Predispatch','Market Demand','Ontario Demand']]

In [49]:
date=pd.to_datetime(df['Date'])
hour=df['Hour']
hour = hour.replace(24,0)
hour=hour.apply(lambda x: datetime.time(x,0))
dt=date.apply(lambda x: str(x.date()))+' '+hour.apply(lambda x: str(x))
dt = pd.to_datetime(dt)
def adjusthour(x):
    if x.hour == 0:
        return x + datetime.timedelta(days=1)
    else:
        return x
dt = dt.apply(adjusthour)

df['Date']=dt
df.drop('Hour',axis=1,inplace=True)

In [50]:
df.rename(columns={'Date':'date','HOEP':'price','Hour 1 Predispatch':'1_hour_pred','Hour 2 Predispatch':'2_hour_pred','Hour 3 Predispatch':'3_hour_pred','Market Demand':'market_demand','Ontario Demand':'ontario_demand'},inplace=True)

In [51]:
df['1_lag_pred']=df['1_hour_pred'].shift()

In [52]:
df['2_lag_pred']=df['2_hour_pred'].shift(2)

In [53]:
df['3_lag_pred']=df['3_hour_pred'].shift(3)

In [54]:
df

,date,price,1_hour_pred,2_hour_pred,3_hour_pred,market_demand,ontario_demand,1_lag_pred,2_lag_pred,3_lag_pred
0,2020-01-01 01:00:00,0.00,0.00,0.00,0.00,16565,13219,NaN,NaN,NaN
1,2020-01-01 02:00:00,0.00,0.00,0.00,0.00,16193,12895,0.00,NaN,NaN
2,2020-01-01 03:00:00,0.00,0.00,0.00,0.00,16072,12554,0.00,0.00,NaN
3,2020-01-01 04:00:00,0.00,0.00,0.00,-0.02,15930,12360,0.00,0.00,0.00
4,2020-01-01 05:00:00,0.00,0.00,0.00,-0.10,15963,12353,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...
26299,2022-12-31 20:00:00,40.90,42.59,42.59,40.99,17872,15834,61.72,57.11,42.18
26300,2022-12-31 21:00:00,46.75,42.34,42.85,41.75,18051,15465,42.59,56.23,55.37
26301,2022-12-31 22:00:00,41.12,41.00,40.16,40.15,17362,15086,42.34,42.59,52.46
26302,2022-12-31 23:00:00,37.95,40.12,40.07,40.12,16277,14453,41.00,42.85,40.99


In [55]:
df.to_csv('final_data.csv')